In [1]:
#Last Updated - Jan 9, 11:AM

In [59]:
import traceback

In [60]:
import ta

In [61]:
class Startegy_2():
    def __init__(self,df_15m,df_5m,df_1m,stock_name,target_ratio,trailing_SL = False):
        
        
        self.df_1m = df_1m
        self.df_5m = df_5m
        self.df_15m = df_15m
        self.df_5m = self.add_vwap()
        self.trailing_SL = trailing_SL
        
        # Set the time range for the strategy (9.15 - 9.45 candles)
        self.start_time = datetime.time(9, 15)
        self.end_time = datetime.time(9, 45)
        self.force_close_time = datetime.time(15, 15)
        
        self.pct_change_threshold = 0.5 #9.45 close w.r.t 9.15 opening
        self.ref_candle_pct_change_threshold = 0.5 #w.r.t 9.15 opening
        self.entry_point_pct_change_threshold = 0.025 #w.r.t REF candle's High
        self.SL_threshold = 0.05 #w.r.t REF candle's LOW
        self.trailing_SL_multiplier = 1
        self.ADX_threshold = 25
        
        
        self.conditions_bool = []
        self.entry_conditions_list = []
        self.logs = []
        self.REF_CANDLE = None
        self.first_entry_point = None
        self.entry_point = False
        self.exit_point = False
        self.PnL = None
        
        self.stop_loss = None
        self.trade_day = True
        
        self.stock_name = stock_name
        self.stop_loss_hit_points = None
        self.target_hit_points = None
        self.target = None
        self.post_entry_df_1m  = None
        self.target_ratio = target_ratio
        
    
  
    def get_report(self):
        date = self.df_1m.day.iloc[0]
        stock_name = self.stock_name

        SL = self.stop_loss
        trade_day = self.trade_day
        PnL = self.PnL
        buffers = {  "pct_change_threshold": self.pct_change_threshold, #9.45 close w.r.t 9.15 opening
                   "ref_candle_pct_change_threshold": self.ref_candle_pct_change_threshold,  #w.r.t 9.15 opening
                   "entry_point_pct_change_threshold": self.entry_point_pct_change_threshold,  #w.r.t REF candle's High
                    "SL_threshold": self.SL_threshold,  #w.r.t REF candle's LOW
                   "trailing_SL_multiplier": self.trailing_SL_multiplier
        }
        
        if type(self.entry_point) != pd.Series:
            entry_time = None
            entry_price = None
            ref_candle_time = None
             
        else:
            entry_time = self.entry_point["Time"]
            entry_price = self.entry_point['buy_price']
            ref_candle_time = self.REF_CANDLE.Time
        
        if type(self.exit_point) != pd.DataFrame: 
            exit_time = None
            exit_price = None
        else:
            exit_time = self.exit_point['Time'].iloc[0]
            exit_price = self.exit_point['Close'].iloc[0]
        
        
        
        report_df =  pd.DataFrame([{"date":date,
                "stock_name":stock_name,
                "ref_candle_time":ref_candle_time,
                 "entry_time":entry_time,
                "entry_price":entry_price,
                "SL":SL,
                 "exit_time": exit_time,
                "exit_price":exit_price,
                "trade_day":trade_day,
                "PnL":PnL,
                 "pct_change_threshold": self.pct_change_threshold, #9.45 close w.r.t 9.15 opening
                   "ref_candle_pct_change_threshold": self.ref_candle_pct_change_threshold,  #w.r.t 9.15 opening
                   "entry_point_pct_change_threshold": self.entry_point_pct_change_threshold,  #w.r.t REF candle's High
                    "SL_threshold": self.SL_threshold,  #w.r.t REF candle's LOW
                   "trailing_SL_multiplier": self.trailing_SL_multiplier,
                    "logs" : self.logs
                 }])
        
        
        return report_df
    
        
    def add_vwap(self):
        df_5m = self.df_5m.copy()
        df_5m['vwap'] = (df_5m['Close'] * df_5m['Volume']).cumsum() / df_5m['Volume'].cumsum()
        return df_5m
    
        
    @staticmethod
    def check_entry_condition_1(row):
        # Check if each candle till 9.45 is completely above the indicator or on the indicator
        if row['High'] > row['vwap']:
            return True
        else:
            return False
        
    def check_entry_condition_2(self,pct_change):
        if pct_change >= self.pct_change_threshold:
            self.logs.append(f"{self.pct_change_threshold}% Threshold reached")
#             print(f"{pct_change_threshold}% Threshold reached")

            return True

        else :
#             print(f"{pct_change_threshold}% Threshold NOT reached")
            self.logs.append(f"{self.pct_change_threshold}% Threshold NOT reached")
            self.trade_day = False
            return False #CHANGED BOOL

    def get_pct_change_1(self):
        df_5m = self.df_5m
        start_time = self.start_time
        end_time = self.end_time
        
        temp_df = df_5m[(df_5m['Time'] >= start_time) & (df_5m['Time'] <= end_time)]
        start_row = temp_df[temp_df['Time'] == start_time]
        end_row = temp_df[temp_df['Time'] == end_time]
        pct_change = ( end_row['Close'].iloc[0] -  start_row['Open'].iloc[0] )*100 / start_row['Open'].iloc[0]

        return pct_change

    #Condition1 for REF
    def REF_touch_and_close_above_vwap(self): 
        
        df_5m = self.df_5m
        #Should touch VWAP, but close only above VWAP
        post_945_df_5m = df_5m[df_5m['Time'] > datetime.time(9,45)]
        #Low should be below or equal to VWAP + High should be above VWAP

        condition_satisfied_df = post_945_df_5m[(post_945_df_5m['Low']<=post_945_df_5m['vwap']) & 
                   (post_945_df_5m['Close']>post_945_df_5m['vwap'])]

        if len(condition_satisfied_df) >0:
            self.logs.append(f"REF_touch_and_close_above_vwap : Passed")         
    #         first_met_point = condition_satisfied_df.iloc[0]
#             print("REF_touch_and_close_above_vwap")
            return True, condition_satisfied_df

        else:
            self.logs.append(f"REF_touch_and_close_above_vwap : Didn't pass")         
            self.trade_day = False
            return False, None

    #Helper for REF
    def get_pct_change_2(self,close_of_condition_satisfied_point):
        df_5m = self.df_5m
        
        open_915 = df_5m[df_5m['Time'] == self.start_time]['Open'].iloc[0]
        pct_change = (close_of_condition_satisfied_point - open_915)*100 / open_915

        return pct_change

    #Condition2 for REF
#     @staticmethod
    def REF_close_greater_than_915_opening(self,condition_satisfied_point,open_915,ref_candle_pct_change_threshold):
        close_of_condition_satisfied_point = condition_satisfied_point['Close']
        pct_change_2 = self.get_pct_change_2(close_of_condition_satisfied_point)

        if pct_change_2 >= ref_candle_pct_change_threshold:
            self.logs.append("ref_candle_pct_change_threshold reached")
            return condition_satisfied_point

        else:
#             print("ref_candle_pct_change_threshold NOT reached")
            self.trade_day = False
            return None #CHANGED this to None from condition_satisfied_point
    
    def check_945_to_entry_candle(self):
        """
        9.45 to Entry candle : High of candles between them should not be less than VWAP 
        """
        if type(self.REF_CANDLE) == pd.Series:
        
            candles_945_to_entry = self.df_5m[(df_5m['Time'] > datetime.time(9,45)) & (df_5m['Time'] < self.entry_point['Time'])]
            candles_not_satsifying_points = candles_945_to_entry[candles_945_to_entry["High"] < candles_945_to_entry["vwap"]]

            if len(candles_not_satsifying_points)!=0:
                self.trade_day = False
                self.logs.append("check_945_to_entry_candle >= VWAP FAILED")
                return False
            else:
                self.logs.append("check_945_to_entry_candle >= VWAP PASSED")
                return True
        

    def get_ref_candle(self):
        
        df_5m = self.df_5m
        
        open_915 = df_5m[df_5m['Time'] == self.start_time]['Open'].iloc[0]
        condition_satisfied_point_bool, condition_satisfied_points = self.REF_touch_and_close_above_vwap()
        if condition_satisfied_point_bool == True:
            for i,each_row in condition_satisfied_points.iterrows():
                REF_POINT = self.REF_close_greater_than_915_opening(each_row, open_915,self.ref_candle_pct_change_threshold )
                if self.trade_day:
                    if all(REF_POINT != None):
                        self.logs.append("REF_CANDLE Exists")    
                        return REF_POINT
            
            self.logs.append("ref_candle_pct_change_threshold NOT reached")
            
            
    def get_ref_candle_v2(self):
        """
        After 945, If open = low = VWAP where close > open, we need to consider even this for ref candle
        """
        
        df_5m = self.df_5m
        post_945_5m = df_5m[df_5m['Time'] > self.end_time]
        mask = (post_945_5m['Open'].eq(post_945_5m['Low'])) \
                & (post_945_5m['Close'].eq(post_945_5m['vwap'])\
                & ( post_945_5m["Close"] > post_945_5m["Open"] ) 
                  )

        if len(post_945_5m[mask])!=0: #If there exists any points with above mask conditions,
            if post_945_5m["Time"].iloc[0] < self.REF_CANDLE.Time:
                self.REF_CANDLE = post_945_5m.iloc[0]
                self.logs.append("REF_candle_v2 point Exists")
                
    def check_if_915_945_green(self):
        my_candles = self.df_15m[ ( self.df_15m['Time'] >= self.start_time ) & ( self.df_15m['Time'] < self.end_time ) ]

        if len(my_candles[my_candles['Close'] > my_candles['Open']])!= len(my_candles):
            self.trade_day = False
            self.logs.append("All candles between 9.15 to 9.45 are not green")
            failed_candles = list(my_candles[~(my_candles['Close'] > my_candles['Open'])]['Time'])
            self.logs.append(f"{failed_candles} are the failed candles")
            
        else:
            self.logs.append("All candles between 9.15 to 9.45 are Green")
            



    def check_entry_conditions(self):
        df_5m = self.df_5m
        
        for i, row in df_5m[(df_5m['Time'] >= self.start_time) & (df_5m['Time'] <= self.end_time)].iterrows():
            self.entry_conditions_list.append(self.check_entry_condition_1(row))

        #Cheking if all the points are meeting condition 1 and storing in the bool_list
        if all(self.entry_conditions_list): 
            self.conditions_bool.append({"9.15 - 9.45 are above the VWAP":True})
            self.logs.append("All candles from 9.15 - 9.45 are above the VWAP")
#             print("All candles from 9.15 - 9.45 are above the VWAP")
        else : 
            self.trade_day = False
            self.conditions_bool.append({"9.15 - 9.45 are above the VWAP":False}) #CHANGED BOOL from True
            self.logs.append("All candles from 9.15 - 9.45 are NOT above the VWAP")
#             print("All candles from 9.15 - 9.45 are NOT above the VWAP")

        #Cheking if the all the points are meeting condition 1 and continuing the process
        #Process is to check condition 2 and push the bool to bool_list
        if self.conditions_bool[0]['9.15 - 9.45 are above the VWAP']: 
            pct_change = self.get_pct_change_1()
            condition_2_bool = self.check_entry_condition_2(pct_change)
            self.conditions_bool.append({"pct_change_threshold_reached":condition_2_bool})
            
            
            #Condition for checking if_915_945 candles are green

#             if self.trade_day:
#                 self.check_if_915_945_green()
                
 
            #Checking if condition 2 is satisfied and continuing the process
            #Process is to get the reference point
            if self.conditions_bool[1]["pct_change_threshold_reached"] & self.trade_day:
                self.REF_CANDLE = self.get_ref_candle()
                self.get_ref_candle_v2() # Checks if open = low = VWAP where close > open happens before current REF and assigns the REF to  respective point



                #return REF_CANDLE , #[conditions_bool,entry_conditions_list]
            else:
                self.trade_day = False
        
        else:
            self.trade_day = False

    def check_ADX(self):
        entry_time = self.entry_point['Time']
        
        if (self.df_5m[self.df_5m['Time'] == entry_time]["adx"]>=self.ADX_threshold).values[0] == True:
            self.logs.append("ADX Threshold Passed")
        else:
            self.trade_day = False
            self.logs.append("ADX Threshold Failed")
            
    
    def get_entry_point(self):
        
        if self.trade_day:
        
            df_1m = self.df_1m
            REF_CANDLE = self.REF_CANDLE


            #Filtering data to be only after REF candle
            after_ref_candle_data_1m = df_1m[df_1m['Time'] > REF_CANDLE['Time']]

            #Checking for entry point condition
            ref_high = REF_CANDLE['High']

            #Below Gives ref_high added with buffer
            ref_high_with_buffer = REF_CANDLE['High'] * ( (self.entry_point_pct_change_threshold)/100 + 1 )  

            #CHANGED Below First line to greater than from less than
            entry_points = after_ref_candle_data_1m[after_ref_candle_data_1m['High'] > ref_high_with_buffer]
            entry_points = entry_points[entry_points['Time'] < datetime.time(14,0)]

            if len(entry_points)>0:
                first_entry_point = entry_points.iloc[0].copy()
                first_entry_point["buy_price"] = ref_high_with_buffer
                self.entry_point = first_entry_point
                self.logs.append("entry point after REF candle Found")
                self.check_945_to_entry_candle() #This function sets the trade_day = False, if condition doesn't satisfy
                self.check_ADX()
        

            else:
                self.trade_day = False
                self.logs.append("entry point after REF candle Not Found")
                
 
    def get_stop_loss(self):

        self.stop_loss = self.REF_CANDLE.Low - (  self.REF_CANDLE.Low * self.SL_threshold /100 )
    
    def get_target_hit_points(self):
        
        entry_price = self.entry_point['buy_price']
        if self.target !=0:
            self.target =  entry_price + (self.target_ratio * ( entry_price -  self.stop_loss ) ) 
            self.target_hit_points = self.post_entry_df_1m[self.post_entry_df_1m['High'] >= self.target]
        else:
            self.target_hit_points = pd.DataFrame([])
            
            
        
    def get_exit_point(self):
        
        if self.trade_day:
        
            self.get_stop_loss()

            if self.trailing_SL == False:
                self.post_entry_df_1m = self.df_1m[ self.df_1m['Time'] > self.entry_point['Time'] ]
                self.stop_loss_hit_points = self.post_entry_df_1m[self.post_entry_df_1m['Low'] <= self.stop_loss]
                
                
                self.get_target_hit_points()


                
                if ( len(self.stop_loss_hit_points) == 0 ) & (len(self.target_hit_points) == 0):
                    force_close_df = df_1m[df_1m['Time'] == self.force_close_time]
                    self.exit_point = force_close_df
                    self.logs.append("Force closed")
            
                elif len(self.target_hit_points) == 0: #target is not hit, but SL is hit
                    self.exit_point = self.stop_loss_hit_points.iloc[:1]
                    self.logs.append("Stop Loss Hit")
                
                elif len(self.stop_loss_hit_points) == 0: #target is hit, but SL is not hit
                    self.exit_point = self.target_hit_points.iloc[:1]
                    self.logs.append("Target Hit")

                else: #Both Target and SL got hit
                    if obj.target_hit_points.iloc[0]['Time'] > obj.stop_loss_hit_points.iloc[0]['Time']:
                        obj.exit_point = obj.stop_loss_hit_points.iloc[:1] #SL Hit first
                        self.logs.append("Stop Loss Hit")

                    else: #Target hit first
                        obj.exit_point = obj.target_hit_points.iloc[:1] #SL Hit first
                        self.logs.append("Target Hit")

    #         else:
    #             self.get_trailing_stop_loss_exit_point()
    
    
            
                
#     def get_trailing_stop_loss_exit_point():
        
#         df_1m = self.df_1m
#         entry_point = self.entry_point

#         #Getting PnL for every minute data
#         post_entry_df_1m = df_1m[df_1m['Time'] > entry_point['Time']].copy()
#         post_entry_df_1m.loc[:,"PnL"] = entry_point['buy_price'] - post_entry_df_1m.loc[:,"Low"]

#         trailing_threshold = ( entry_point['buy_price'] - stop_loss ) * self.trailing_SL_multiplier

#         #If there's any where we meet this condition for trailing, then make the SL as buyprice
#         trailing_SL_df = post_entry_df_1m[post_entry_df_1m['PnL'] >= self.trailing_threshold]

#         if len(trailing_SL_df) == 0 :
#             #There's no point so that we can trail the SL
#             pass
#         else:
#             #We can trail the SL. So write code here for executing it.
#             self.stop_loss = entry_point.buy_price

    def get_PnL(self):
        if self.trade_day:
            sell_price = self.exit_point["Close"].iloc[0]
            buy_price = self.entry_point['buy_price']
            self.PnL = sell_price - buy_price
        



    def backtest(self):
        self.check_entry_conditions()
        self.get_entry_point()
        self.get_exit_point()
        self.get_PnL()
        
        

            
            
        

    

In [62]:
def get_report(df, stock_name, comments):
    total_days = len(df)
    total_trade_days = len(df[df["trade_day"]==True])

    trade_days = df[df["trade_day"]==True]
    non_loss_days = df[df['PnL']>=0]
    profit_days = df[df['PnL']>0]
    loss_days = df[df['PnL']<0]

    profit_proba = len(profit_days) / total_trade_days
    max_profit = profit_days['PnL'].max()
    max_loss = loss_days['PnL'].min()

    net_PnL = trade_days['PnL'].sum()

    return pd.DataFrame([{"stock_name":stock_name,
            "total_days":total_days,
            "total_trade_days":total_trade_days,
            "profit_days":len(profit_days),
            "loss_days":len(loss_days),
            "profit_probability": profit_proba,
            "max_profit":max_profit,
            "max_loss":max_loss,
            "net_PnL": net_PnL,
            "comments":comments
           }])

## Start

In [64]:
import os

In [65]:
os.listdir("data/")

['NSE:BALKRISIND-EQ_5.csv',
 'NSE:TATAMOTORS-EQ_1.csv',
 'NSE:TATAMOTORS-EQ_5.csv',
 'NSE:BALKRISIND-EQ_1.csv',
 'NSE:ADANIENT-EQ_15.csv',
 'NSE:ADANIENT-EQ_60.csv',
 'NSE:ADANIENT-EQ_1.csv',
 'NSE:TATAMOTORS-EQ_15.csv',
 'NSE:ADANIENT-EQ_5.csv',
 'NSE:ADANIENT-EQ_30.csv']

In [66]:
import pandas as pd
import datetime

base_data_path = "data"

## Data Prep
stock_name = "NSE:ADANIENT"

# Load the data for the stock, including both the 5 minute and 1 minute time frame data
df_15m_all = pd.read_csv(f"{base_data_path}/{stock_name}-EQ_15.csv", parse_dates = ['Time'])
df_5m_all = pd.read_csv(f"{base_data_path}/{stock_name}-EQ_5.csv", parse_dates = ['Time'])
df_1m_all = pd.read_csv(f"{base_data_path}/{stock_name}-EQ_1.csv", parse_dates = ['Time'])

df_15m_all['day'] = df_15m_all.Time.dt.date
df_15m_all['Time'] = df_15m_all.Time.dt.time

df_5m_all['day'] = df_5m_all.Time.dt.date
df_5m_all['Time'] = df_5m_all.Time.dt.time

df_1m_all['day'] = df_1m_all.Time.dt.date
df_1m_all['Time'] = df_1m_all.Time.dt.time

In [68]:
def add_adx(df_5m):
    df_5m['adx'] = ta.trend.adx(df_5m["High"],df_5m["Low"],df_5m["Close"])
    return df_5m

In [69]:
df_5m_all_with_adx = add_adx(df_5m_all)

/Users/porteruser/opt/anaconda3/lib/python3.9/site-packages/ta/trend.py:780: RuntimeWarning: invalid value encountered in double_scalars
  dip[idx] = 100 * (self._dip[idx] / value)
/Users/porteruser/opt/anaconda3/lib/python3.9/site-packages/ta/trend.py:785: RuntimeWarning: invalid value encountered in double_scalars
  din[idx] = 100 * (self._din[idx] / value)


In [70]:
from tqdm import tqdm

In [25]:
all_dates_list = df_5m_all['day'].unique()
report_df = pd.DataFrame([])
objs_list = []
errors_list = []
target_ratio = 2


for i in tqdm(range(len(all_dates_list))):
 
    try:
        df_15m = df_15m_all[df_15m_all['day'] == all_dates_list[i]]
        df_5m = df_5m_all[df_5m_all['day'] == all_dates_list[i]]
        df_1m = df_1m_all[df_1m_all['day'] == all_dates_list[i]]

        obj = Startegy_2(df_15m,df_5m,df_1m,stock_name,target_ratio)
        obj.backtest()
        current_report = obj.get_report()
        report_df = pd.concat([report_df,current_report])

        if obj.trade_day == True:
            pass
#             print(i)

        objs_list.append(obj)
    except:
        errors_list.append(i)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1289/1289 [00:39<00:00, 32.35it/s]


In [44]:
comments = f"strategy = VWAP, target_ratio = {target_ratio}"

In [11]:
# report_df.to_csv(f"{stock_name}_VWAP_with_1:{target_ratio}_target.csv",index = False)

## Testing

In [71]:
all_dates_list = df_5m_all['day'].unique()

In [72]:
my_date = all_dates_list[867]

In [73]:
df_15m = df_15m_all[df_15m_all['day'] == my_date]
df_5m = df_5m_all[df_5m_all['day'] == my_date]
df_1m = df_1m_all[df_1m_all['day'] == my_date]

obj = Startegy_2(df_15m,df_5m,df_1m,"TATAMOTORS-EQ",0)
obj.backtest()
report = obj.get_report()

# report

In [74]:
df_15m = obj.df_15m
# post_945_5m = df_5m[(df_5m['Time'] > obj.end_time) & (df_5m['Time'] < obj.entry_point['Time'])]

In [75]:
my_candles = obj.df_15m[ ( obj.df_15m['Time'] >= obj.start_time ) & ( obj.df_15m['Time'] < obj.end_time ) ]

In [76]:
obj.logs

['All candles from 9.15 - 9.45 are above the VWAP',
 '0.5% Threshold reached',
 'REF_touch_and_close_above_vwap : Passed',
 'ref_candle_pct_change_threshold reached',
 'REF_CANDLE Exists',
 'entry point after REF candle Found',
 'check_945_to_entry_candle >= VWAP PASSED',
 'ADX Threshold Passed',
 'Stop Loss Hit']

## QC

In [66]:
# obj.stop_loss_hit_points = obj.target_hit_points.iloc[2:]

In [69]:
obj.target_hit_points.iloc[0]['Time']

datetime.time(15, 5)

In [70]:
obj.stop_loss_hit_points.iloc[0]['Time']

datetime.time(15, 7)

In [72]:
self.df_5m['adx'] = ta.trend.adx(self.df_5m["High"],df_5m["Low"],df_5m["Close"])

In [54]:
# df_15m_all["adx"] = ta.trend.adx(df_15m_all["High"],df_15m_all["Low"],df_15m_all["Close"])

,Time,Open,High,Low,Close,Volume,day,adx
0,09:15:00,138.00,139.45,137.90,139.05,230970,2017-08-01,0.000000
1,09:30:00,139.05,140.80,138.80,140.15,412286,2017-08-01,0.000000
2,09:45:00,140.05,140.40,139.90,140.05,132118,2017-08-01,0.000000
3,10:00:00,140.05,140.20,139.80,139.90,67433,2017-08-01,0.000000
4,10:15:00,139.85,139.90,139.40,139.70,62685,2017-08-01,0.000000
...,...,...,...,...,...,...,...,...
32127,14:15:00,3238.85,3242.45,3231.00,3238.00,34738,2022-10-14,19.657278
32128,14:30:00,3238.00,3238.65,3212.45,3228.90,110768,2022-10-14,19.444957
32129,14:45:00,3227.50,3234.00,3217.00,3217.00,56763,2022-10-14,19.247801
32130,15:00:00,3217.00,3225.45,3205.20,3210.05,128715,2022-10-14,19.836834
